Librerías del Proyecto

In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from nltk.tokenize import word_tokenize
import os
 
import string
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
nltk.download('stopwords')
 
# Importing libraries necessary for Model Building and Training
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
 
import warnings
warnings.filterwarnings('ignore')

import mailbox
from email import policy
from email.parser import BytesParser

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Home\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
archivo_correo = mailbox.mbox(r'E:\Stuff\CUCEI\12vo\SEMINARIO DE INTELIGENCIA ARTIFICIAL II\proyecto\easy_ham\0001.ea7e79d3153e7469e7a9c3e0af6a357e')

with open(r'E:\Stuff\CUCEI\12vo\SEMINARIO DE INTELIGENCIA ARTIFICIAL II\proyecto\easy_ham\0001.ea7e79d3153e7469e7a9c3e0af6a357e', 'rb') as fp:
    archivo_correo_2 = BytesParser(policy=policy.default).parse(fp)

archivo_correo_2.get_payload()


'    Date:        Wed, 21 Aug 2002 10:54:46 -0500\n    From:        Chris Garrigues <cwg-dated-1030377287.06fa6d@DeepEddy.Com>\n    Message-ID:  <1029945287.4797.TMDA@deepeddy.vircio.com>\n\n\n  | I can\'t reproduce this error.\n\nFor me it is very repeatable... (like every time, without fail).\n\nThis is the debug log of the pick happening ...\n\n18:19:03 Pick_It {exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace} {4852-4852 -sequence mercury}\n18:19:03 exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace 4852-4852 -sequence mercury\n18:19:04 Ftoc_PickMsgs {{1 hit}}\n18:19:04 Marking 1 hits\n18:19:04 tkerror: syntax error in expression "int ...\n\nNote, if I run the pick command by hand ...\n\ndelta$ pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace  4852-4852 -sequence mercury\n1 hit\n\nThat\'s where the "1 hit" comes from (obviously).  The version of nmh I\'m\nusing is ...\n\ndelta$ pick -version\npick -- nmh-1.0.4 [compiled on fuchsia.c

In [20]:

for mensaje in archivo_correo:
    print(mensaje.is_multipart())
    cuerpo = mensaje.get_payload(decode=True).decode(errors='replace')
cuerpo

False


'    Date:        Wed, 21 Aug 2002 10:54:46 -0500\n    From:        Chris Garrigues <cwg-dated-1030377287.06fa6d@DeepEddy.Com>\n    Message-ID:  <1029945287.4797.TMDA@deepeddy.vircio.com>\n\n\n  | I can\'t reproduce this error.\n\nFor me it is very repeatable... (like every time, without fail).\n\nThis is the debug log of the pick happening ...\n\n18:19:03 Pick_It {exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace} {4852-4852 -sequence mercury}\n18:19:03 exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace 4852-4852 -sequence mercury\n18:19:04 Ftoc_PickMsgs {{1 hit}}\n18:19:04 Marking 1 hits\n18:19:04 tkerror: syntax error in expression "int ...\n\nNote, if I run the pick command by hand ...\n\ndelta$ pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace  4852-4852 -sequence mercury\n1 hit\n\nThat\'s where the "1 hit" comes from (obviously).  The version of nmh I\'m\nusing is ...\n\ndelta$ pick -version\npick -- nmh-1.0.4 [compiled on fuchsia.c

Abriendo las carpetas de spam y no spam:

In [50]:
os.getcwd()

'e:\\Stuff\\CUCEI\\12vo\\SEMINARIO DE INTELIGENCIA ARTIFICIAL II\\proyecto'

In [97]:
def email_parser(mbox, carp):
    message = []
    for mensaje in mbox:
        cuerpo = ""
        if mensaje.is_multipart():
            # Si el correo tiene varias partes, buscar la parte de texto/html
            message.append("multipart")
            for parte in mensaje.get_payload():
                if parte.get_content_type() == 'text/plain' or parte.get_content_type() == 'text/html':
                    cuerpo = parte.get_payload(decode=True).decode(errors='replace')
                    cuerpo = cuerpo.translate(str.maketrans('', '', string.punctuation))
                    message.append(cuerpo)
                    message.append(carp)
        else:
            # Si el correo es de una sola parte, obtener el cuerpo directamente
            char_set = mensaje.get_charset()
            cuerpo = mensaje.get_payload(decode=True).decode(errors='replace')
            cuerpo = cuerpo.translate(str.maketrans('', '', string.punctuation))
            message.append(cuerpo)
            message.append(carp)

        return message

In [98]:
carpeta = ["spam", "easy_ham"]  # Ruta de la carpeta que deseas leer
message_list = []
for carp in carpeta:
    # Obtener la lista de archivos en la carpeta
    lista_archivos = [f'{os.path.join(os.getcwd(), carp)}\\{x}' for x in os.listdir(os.path.join(os.getcwd(), carp))]
    #print(lista_archivos)
    # Leer cada archivo de la lista
    for archivo in lista_archivos:
        if os.path.isfile(archivo):  # Verificar que sea un archivo
            # Abrir el dataset de correos
            mbox = mailbox.mbox(archivo)
            # Recorrer cada correo en el dataset
            message = email_parser(mbox, carp)
            if message == None:
                print(archivo)
            else:
                message_list.append(message)

e:\Stuff\CUCEI\12vo\SEMINARIO DE INTELIGENCIA ARTIFICIAL II\proyecto\spam\0035.8e582263070076dfe6000411d9b13ce6
e:\Stuff\CUCEI\12vo\SEMINARIO DE INTELIGENCIA ARTIFICIAL II\proyecto\spam\0036.8e582263070076dfe6000411d9b13ce6
e:\Stuff\CUCEI\12vo\SEMINARIO DE INTELIGENCIA ARTIFICIAL II\proyecto\spam\0037.7ce3307b56dd90453027a6630179282e
e:\Stuff\CUCEI\12vo\SEMINARIO DE INTELIGENCIA ARTIFICIAL II\proyecto\spam\0038.7ce3307b56dd90453027a6630179282e
e:\Stuff\CUCEI\12vo\SEMINARIO DE INTELIGENCIA ARTIFICIAL II\proyecto\spam\0039.256602e2cb5a5b373bdd1fb631d9f452
e:\Stuff\CUCEI\12vo\SEMINARIO DE INTELIGENCIA ARTIFICIAL II\proyecto\spam\0040.256602e2cb5a5b373bdd1fb631d9f452
e:\Stuff\CUCEI\12vo\SEMINARIO DE INTELIGENCIA ARTIFICIAL II\proyecto\spam\0041.21cc985cc36d931916863aed24de8c27
e:\Stuff\CUCEI\12vo\SEMINARIO DE INTELIGENCIA ARTIFICIAL II\proyecto\spam\0042.21cc985cc36d931916863aed24de8c27
e:\Stuff\CUCEI\12vo\SEMINARIO DE INTELIGENCIA ARTIFICIAL II\proyecto\spam\0043.8d93819b95ff90bf2e2b141c2

In [99]:
message_list

[['DOCTYPE HTML PUBLIC W3CDTD HTML 40 TransitionalEN\nHTMLHEAD\nMETA contenttexthtml charsetwindows1252 httpequivContentType\nMETA contentMSHTML 50023141000 nameGENERATORHEAD\nBODY Inserted by Calypso \nTABLE border0 cellPadding0 cellSpacing2 idCalyPrintHeader rulesnone \nstyleCOLOR black DISPLAY none width100\n  TBODY\n  TR\n    TD colSpan3\n      HR colorblack noShade SIZE1\n    TDTRTDTR\n  TR\n    TD colSpan3\n      HR colorblack noShade SIZE1\n    TDTRTBODYTABLE End Calypso  Inserted by Calypso FONT \ncolor000000 faceVERDANAARIALHELVETICA size2BRFONTTDTRTABLE End Calypso FONT colorff0000 \nfaceCopperplate Gothic Bold size5 PTSIZE10\nCENTERSave up to 70 on Life InsuranceCENTERFONTFONT colorff0000 \nfaceCopperplate Gothic Bold size5 PTSIZE10\nCENTERWhy Spend More Than You Have To\nCENTERFONT colorff0000 faceCopperplate Gothic Bold size5 PTSIZE10\nCENTERLife Quote Savings\nCENTER\nP alignleftP\nP alignleftPFONTUIBBRFONTUBUI\nPP\nCENTER\nTABLE border0 borderColor111111 cellPadding0 cel